In [15]:
from tensorflow import keras
import tensorflow.keras as keras
from tensorflow.keras import layers
import numpy as np
import sklearn
import sklearn.metrics

In [5]:
# get the data
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")


Because the model export/import apparently does not work, we need a workaround. First create the model architecture  (which we luckily know), then "fit" the model with 0 epochs (i.e. do not fit it at all) and finally, load the weights of the saved model into the empty "frame" of the NN.

In [9]:
input_shape = x_train.shape[1:]
num_classes = 8

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation = "selu"),
        layers.Dense(units=128, activation="relu"),
        layers.Dense(units=128, activation="relu"),
        #layers.Dense(units=128, 
        #      activation=keras.layers.LeakyReLU(alpha=0.05)), 
        # layers.Dense(units=128, 
        #      activation=keras.layers.LeakyReLU(alpha=0.01)), 
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
model.load_weights('KerasNN_FullFinanceData_ReLu.h5')

2021-12-20 16:04:45.688145: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-12-20 16:04:45.745931: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3299990000 Hz
2021-12-20 16:04:45.746387: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3fb0850 executing computations on platform Host. Devices:
2021-12-20 16:04:45.746399: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-12-20 16:04:45.760938: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or se

In [19]:
model.compile(loss="categorical_crossentropy", 
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=["accuracy"])

In [20]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.09173799476404101
Test accuracy: 0.9705713


In [21]:
sklearn.metrics.confusion_matrix(np.argmax(y_test, axis=1),   # true labels
                                 model.predict_classes(x_test)) # predictions

array([[229,   4,   0,   0,   0,   7,   0,   0],
       [  5, 227,   0,   3,   0,   0,   0,   0],
       [  0,   0,  68,   0,   0,   0,   0,   0],
       [  0,   2,   3,  65,   0,   0,   0,   0],
       [  1,   0,   0,   0, 151,   2,   0,   1],
       [  3,   0,   0,   0,   0, 244,   0,   0],
       [  0,   0,   0,   0,   0,   0, 378,   0],
       [  0,   0,   0,   0,   7,   2,  11, 320]])

Very good results!